# Praktyczna analiza danych tekstowych- pakiet informacji z Twittera
Analiza sentymentu

NLP - interdyscyplinarna dziedzina, łącząca zagadnienia sztucznej inteligencji i językoznawstwa, zajmująca się automatyzacją analizy, rozumienia, tłumaczenia i generowania języka naturalnego przez komputer.

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('./Downloads/train_E6oV3lV.csv')

In [ ]:
#Liczenie słów
train['word_count'] = train['tweet'].apply(lambda x: len(str(x).split(" ")))
train[['tweet','word_count']].head()

In [ ]:
#Liczenie znaków
train['char_count'] = train['tweet'].str.len() ## this also includes spaces
train[['tweet','char_count']].head()

In [ ]:
#Średnia długość słowa
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

train['avg_word'] = train['tweet'].apply(lambda x: avg_word(x))
train[['tweet','avg_word']].head()

In [ ]:
#Liczenie hasztagów
train['hastags'] = train['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
train[['tweet','hastags']].head()

In [ ]:
#Liczba znaków numerycznych
train['numerics'] = train['tweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
train[['tweet','numerics']].head()

In [ ]:
#Liczba znaków rozpoczynających się od dużej litery
train['upper'] = train['tweet'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
train[['tweet','upper']].head()

# Basic pre-processing 
#Zamiana słów z dużej na małą

In [ ]:
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train['tweet'].head()

In [ ]:
#Usunięcie punktacji 
train['tweet'] = train['tweet'].str.replace('[^\w\s]','')
train['tweet'].head()

In [ ]:
#Usuniecie common word - czesto wystepujacych słów
freq = pd.Series(' '.join(train['tweet']).split()).value_counts()[:10]
freq

In [ ]:
#Usunięcie pozostalych słów, które nie przydadza sie w procesie
freq = list(freq.index)
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train['tweet'].head()

In [ ]:
#Usunięcie słów rzadkich, których jest za mało, żeby cokolwiek dały w analizie 
freq = pd.Series(' '.join(train['tweet']).split()).value_counts()[-10:]
freq

In [ ]:
! pip install -U textblob
! python -m textblob.download_corpora

In [ ]:
#Poprawa błędów w tekście
from textblob import TextBlob
train['tweet'][:5].apply(lambda x: str(TextBlob(x).correct()))

# Tokenizacja - polega na rozbiciu dokumentu tekstowego na poszczególne wyrazy.

In [ ]:
#Tokenizacja
TextBlob(train['tweet'][1]).words

# Stemming - sprowadzenie danych do ich rdzeni
Przykładowo:
angielskie słowa: „connection”, „connections”, „connective”, „connected”, „connecting” poddane stemmingowi dadzą ten sam wyniki, czyli słowo „connect”
Założenie jest takie, że wynik stemmingu ma być podatny na przyimki, rodzaje etc.

In [ ]:
#Stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
train['tweet'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

# Lematyzacja to sprowadzenie grupy wyrazów stanowiących odmianę danego zwrotu do wspólnej postaci.
Umowżliwia to traktowanie ich wszystkich jako te samo słowo.

In [ ]:
#Lematyzacja
from textblob import Word
train['tweet'] = train['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
train['tweet'].head()

In [ ]:
#Łączenie słów - podmiotu do orzeczenia etc.

TextBlob(train['tweet'][0]).ngrams(2)

In [ ]:
#Częstotliwość słów
tf1 = (train['tweet'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

# Badanie sentymentu

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
train_bow = bow.fit_transform(train['tweet'])
train_bow

In [ ]:
train['tweet'][:5].apply(lambda x: TextBlob(x).sentiment)

In [ ]:
train['sentiment'] = train['tweet'].apply(lambda x: TextBlob(x).sentiment[0] )
train[['tweet','sentiment']].head()

In [ ]:
train[['tweet','sentiment']]